In [1]:
import json
import pandas as pd
import psycopg2
from psycopg2.extensions import ISOLATION_LEVEL_AUTOCOMMIT

In [2]:
#ouverture de la connexion avec postgres

db_params = {
    "database": "job_market",
    "user": "admin",
    "password": "root",
    "host": "localhost",
    "port": "5432"
}

conn = psycopg2.connect(**db_params)
conn.set_isolation_level(ISOLATION_LEVEL_AUTOCOMMIT)
cur = conn.cursor()

In [ ]:

# création du dataframe complet

job_offer_adzuna = pd.read_csv('../output/job_offers_adzuna.csv')
job_offer_wttj = pd.read_csv('../output/job_offers_wttj.csv')

df = pd.concat([job_offer_adzuna, job_offer_wttj], ignore_index=True)

# nettoyage rapide

df = df[~df.company.isna()].reset_index(drop=True)

#print(f"type of df.starting_date {type(df.starting_date)} et df.starting_date {df.starting_date} \n")

df = df.assign(
	#starting_date=pd.to_datetime(df.starting_date,dayfirst=True)
    starting_date=pd.to_datetime(df['starting_date'], format='%d %B %Y')
)
df['starting_date'] = df['starting_date'].dt.date

df['publication_date'] = pd.to_datetime(df['publication_date'], errors='coerce', format='%Y-%m-%d')

df = df.where(pd.notnull(df), None)

In [49]:
# for the Skills table
# attention au R pour qu'il ne soit pas confondu avec une majuscule

df_skills = pd.read_csv('./skills.csv')
skills_list = df_skills.iloc[:, 0].tolist()
#print(list(skills_list))
#print(len(skills_list))
#print(type(skills_list))

for skill in skills_list:
    cur.execute("SELECT skillName FROM Skills WHERE skillName = %s;", (skill,))
    result = cur.fetchone()
    if result is None:
        cur.execute("INSERT INTO Skills (skillName) VALUES (%s);", [skill])

['agile', 'airbyte', 'airflow', 'amazon s3', 'amplitude', 'anglais', 'ansible', 'api', 'athena', 'aws', 'aws athena', 'aws glue', 'aws lambda', 'aws s3', 'azure data factory', 'azure databricks', 'bases de données relationnelles', 'bash', 'beam', 'big query', 'blob', 'braze', 'cassandra', 'ci/cd', 'clickhouse', 'cloud functions', 'cloudera', 'code idempotency', 'code quality', 'concourse', 'confluence', 'continuous delivery', 'dash', 'dask', 'data/business intelligence (bi)', 'databricks', 'dataflow', 'dataiku', 'datamesh', 'dataops', 'datastage', 'datastudio', 'dbt', 'denodo', 'devops', 'docker', 'dremio', 'dynamodb', 'elastic compute cloud (ec2)', 'elastic container service (ecs)', 'elastic kubernetes service', 'elasticsearch', 'elk', 'elt', 'fastapi', 'flask', 'flink', 'flume', 'gcp', 'gcp bigquery', 'gcp cloud build', 'gcp cloud run', 'git', 'gitlab', 'glue', 'go', 'google bigquery', 'gradle', 'hadoop', 'hbase', 'hdfs', 'helm', 'hive', 'hudi', 'impala', 'influxdb', 'intellij', 'jav

In [52]:
"""
AVANT 
for the JobOffer_Skills table
2 possiblités : 
    soit une ligne par skillId (et donc plusieurs lignes pour le même jobId) 
    soit une ligne par jobId  ligne (et donc plusieurs pour le même skillId)
Un offre d’emploi peut être associée à plusieurs compétences et vice versa.

pour rappel
CREATE TABLE JobOffer_Skills (
        jobOfferId INTEGER REFERENCES JobOffers,
        skillId INTEGER REFERENCES Skills(skillId),
        PRIMARY KEY (jobOfferId, skillId)
    );


POURQUOI PAS ajouter skillId dans JOboffers directement ?
"""
def get_or_create_skill(cur, description):
    skillids_in_offer = []
    print(description, "\n")
    for skill in skills_list:
        if skill in description:
            print(f"OMG_____   skill {skill}____________\n\n\n\n\n")
            cur.execute("SELECT skillId FROM Skills WHERE skillName = %s;", [skill])
            result = cur.fetchone()
            skillids_in_offer.append(result[0])
    return [skillids_in_offer]
    """if result:
            # il faut lier le skillId (où je le trouve ?) au JobId (où je le trouve ?)
            cur.execute("INSERT INTO JobOffer_Skills (sourceName) VALUES (%s);", (source_name,))
    """

In [43]:
def get_or_create_company(cur, company_data):
    company = eval(company_data)
    cur.execute("SELECT companyId FROM Companies WHERE companyName = %s;", (company['name'],))
    result = cur.fetchone()
    if result:
        return result[0]
    else:
        cur.execute("INSERT INTO Companies (companyName, location, sector, information) VALUES (%s, %s, %s, %s) RETURNING companyId;",
                    (company['name'], company.get('location'), company.get('sector'), ''))
        return cur.fetchone()[0]

In [46]:
def get_or_create_source(cur, source_name):
    cur.execute("SELECT sourceId FROM Source WHERE sourceName = %s;", (source_name,))
    result = cur.fetchone()
    if result:
        return result[0]
    else:
        cur.execute("INSERT INTO Source (sourceName) VALUES (%s) RETURNING sourceId;", (source_name,))
        return cur.fetchone()[0]

In [ ]:
for _, row in df.iterrows():
    company_id = get_or_create_company(cur, row['company'])
    source_id = get_or_create_source(cur, row['source'])
    skill_id = get_or_create_skill(cur, row['description'])

    job_offer_data = (
        row['title'],
        company_id,
        row['salary'],
        row['remote_type'],
        row['contract_type'],
        row['starting_date'] if not pd.isnull(row['starting_date']) else None,
        row['location'],
        row['require_experience'],
        row['education'],
        row['description'],
        row['profil_experience'],
        row['publication_date'].date() if not pd.isnull(row['publication_date']) else None,
        row['url_direct_offer'],
        source_id,
        skill_id
    )
    cur.execute("INSERT INTO JobOffers (title, companyId, salary, remoteType, contractType, startingDate, location, requiredExp, education, descriptions, profilExp, publicationDate, jobLink, sourceId, skillId) VALUES (%s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s);", job_offer_data)

['agile', 'airbyte', 'airflow', 'amazon s3', 'amplitude', 'anglais', 'ansible', 'api', 'athena', 'aws', 'aws athena', 'aws glue', 'aws lambda', 'aws s3', 'azure data factory', 'azure databricks', 'bases de données relationnelles', 'bash', 'beam', 'big query', 'blob', 'braze', 'cassandra', 'ci/cd', 'clickhouse', 'cloud functions', 'cloudera', 'code idempotency', 'code quality', 'concourse', 'confluence', 'continuous delivery', 'dash', 'dask', 'data/business intelligence (bi)', 'databricks', 'dataflow', 'dataiku', 'datamesh', 'dataops', 'datastage', 'datastudio', 'dbt', 'denodo', 'devops', 'docker', 'dremio', 'dynamodb', 'elastic compute cloud (ec2)', 'elastic container service (ecs)', 'elastic kubernetes service', 'elasticsearch', 'elk', 'elt', 'fastapi', 'flask', 'flink', 'flume', 'gcp', 'gcp bigquery', 'gcp cloud build', 'gcp cloud run', 'git', 'gitlab', 'glue', 'go', 'google bigquery', 'gradle', 'hadoop', 'hbase', 'hdfs', 'helm', 'hive', 'hudi', 'impala', 'influxdb', 'intellij', 'jav

In [ ]:
"""
Direction des flèches: il y a deux relations impliquées ici
JobOffer vers JobOffer_Skills
Skill vers JobOffer_Skills
Signification : Chaque flèche pointe vers JobOffer_Skills, indiquant qu’elle sert de table de jonction. 
Il n’y a pas de flèche directe entre JobOffer et Skills parce que leur relation est médiée à travers JobOffer_Skills. 
Un offre d’emploi peut être associée à plusieurs compétences et vice versa.
"""


In [39]:
#couper la connexion

conn.commit()
cur.close()
conn.close()